In [176]:
from typing import Union, Type, Tuple, Optional
from dataclasses import dataclass
from math import ceil, floor
from copy import deepcopy
import re

@dataclass
class Pair:
    left : "Optional[Type[Pair]]"
    right : "Optional[Type[Pair]]"
    value : "Optional[int]"
    parent : "Optional[Type[Pair]]"
    
    def type(self):
        if (self.value==None):
            assert(self.left is not None and self.right is not None)
            return "node"
        else: 
            assert(self.left is None and self.right is None)
            return "leaf"
        
    def magnitude(self):
        if self.type()=="node":
            return 3 * self.left.magnitude() + 2 * self.right.magnitude()
        else:
            return self.value
            
    @staticmethod    
    def newNode(left: "Union[Type[Pair], int]", right :"Union[Type[Pair], int]", parent : "Optional[Type[Pair]]"= None):
        p = Pair(left,
                 right,
                 None, 
                 parent)                 
        p.left.parent = p
        p.right.parent = p
        return p
        
    @staticmethod    
    def newLeaf(value: int, parent: "Optional[Type[Pair]]" = None):
        return Pair(None,
                 None,
                 value, 
                 parent)
       
    def __add__(self, other :"Type[Pair]"):
        s = Pair.newNode(self, other)
        res = "start"
        while res:
            res = s._reduce()
        return s

    def _reduce(self, nesting_level=0):
        reduced = True
        while reduced:
            reduced = False
            nodes = []
            Pair.walk_tree(self, nodes, 0)       
            node_to_split = None
            for node, nesting_level in nodes:
                if node.type()=="node" and nesting_level == 4:
                    node._explode()
                    reduced = True
                    break
                if node.type() == "leaf":
                    if node.value >= 10 and node_to_split is None:
                        node_to_split = node
            if not reduced and node_to_split is not None:
                node_to_split._split()
                reduced = True
            
    @staticmethod
    def walk_tree(root: "Type[Pair]", q, nesting_level):  
        q.append((root, nesting_level))
        if root.type()=="node":
            # First recur on left child
            Pair.walk_tree(root.left, q, nesting_level+1)
            # the recur on right child
            Pair.walk_tree(root.right, q, nesting_level+1)  
        
        
    def _split(self):
        assert(self.type()=="leaf")
        div2 = self.value / 2
        self.left = Pair.newLeaf(floor(div2), self)
        self.right = Pair.newLeaf(ceil(div2), self) 
        self.value = None
    
    def _explode(self):
        assert(self.type()=="node" and self.left.type()=="leaf" and self.right.type()=="leaf")
        l = self.find_first_leaf_left()
        r = self.find_first_leaf_right()
        if l is not None:
            l.value += self.left.value
        if r is not None:
            r.value += self.right.value
        self.value = 0
        self.left = None
        self.right = None
        
    def find_first_leaf_left(self):
        if self.type() == "leaf":
            return self
        elif self.parent is not None:
            is_right_node = id(self) == id(self.parent.right)
            if (is_right_node):
                r = self.parent.left.find_first_leaf_left_in_subtree()
                if r is not None: 
                    return r
            else:
                r = self.parent.find_first_leaf_left()
                if r is not None:
                    return r
                    
    def find_first_leaf_right(self):
        if self.type() == "leaf":
            return self
        elif self.parent is not None:
            is_left_node = id(self) == id(self.parent.left)
            if is_left_node:
                r = self.parent.right.find_first_leaf_right_in_subtree()
                if r is not None: 
                    return r
            else:
                r = self.parent.find_first_leaf_right()
                if r is not None: 
                    return r
    
    def find_first_leaf_right_in_subtree(self):
        if self.type() == "leaf":
            return self
        r = self.left.find_first_leaf_right_in_subtree()
        if r is not None:
            return r 
        return self.right.find_first_leaf_right_in_subtree()
    
    def find_first_leaf_left_in_subtree(self):
        if self.type() == "leaf":
            return self
        r = self.right.find_first_leaf_left_in_subtree()
        if r is not None:
            return r 
        return self.left.find_first_leaf_left_in_subtree()
                            
    def __repr__(self):
        if self.type()=="leaf":
            return str(self.value)
        else:
            s = "["
            s += self.left.__repr__()
            s += ","
            s += self.right.__repr__()
            s += "]"
            return s
        
    
def parse_internal_number(s: str, pos: int) -> Tuple[Union[int, Pair], int]:
    if s[pos]=="[":
        l, new_pos = parse_internal_number(s, pos + 1)
        r, new_pos = parse_internal_number(s, new_pos + 1)
        n = Pair.newNode(l ,r)
        new_pos += 1
    else:
        n_str = re.match("(\d+)*", s[pos:]).groups()[0]
        n = Pair.newLeaf(int(n_str))
        new_pos = pos + len(n_str)
    return n, new_pos

    
def parse_number(s: str) -> Pair:
    return parse_internal_number(s, 0)
    
with open("input.txt", "r") as f:
   numbers = list(map(lambda l: parse_number(l)[0], f.read().splitlines()))


p = deepcopy(numbers[0])
for n in numbers[1:]:    
    p = p + deepcopy(n)
    
answer_1 = p.magnitude()

answer_2 = 0
for i in range(len(numbers)):
    for j in range(len(numbers)):
        if i!=j:
            m = (deepcopy(numbers[i])+deepcopy(numbers[j])).magnitude()
            answer_2 = max(answer_2, m)
            
print(f"answer_1: {answer_1}")
print(f"answer_2: {answer_2}")



answer_1: 4391
answer_2: 4626
